`StructType` and `StructField` With Examples

PySpark `StructType` and `StructField` classes are used to programmatically specify the schema to
the DataFrame and create complex columns like nested struct, array, and map columns.
`StructType` is a collection of `StructField` that defines column name, column data type, boolean 
to specify if the field can be nullable or not metadata. Though PySpark infers the schema from the 
data, sometimes we may need to define our own column names and data types and this article explains
how to define simple, nested and complex schemas

1. `StructType` defines the structure of the DataFrame

PySpark provides from `pyspark.sql.types import StructType` class to define the structure of a DataFrame.
`StructType` is a collection or list of `StructField` objects. `StructType` is a collection or list of `StructField` objects.

2. `StructField` defines the metadata of the `DataFrame` column

`PySpark` provides `pyspark.sqltypes import StructField` class to define the columns which include column name
(`String`), column type (`DataType`), nullable column (`Boolean`) and metadata (`MetaData`)